# Laboratorium Analiza i bazy danych

## Łączenie tabel, podzapytania i funkcje agregujące



### **Imię i Nazwisko:** Kacper Moździerz

### **Numer albumu:** 401733

### **Data ćwiczeń:** 22.10.2021r., 13:00 - 14:30

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

# Wykonanie zadań wprowadzających:

In [1]:
import main as m
import numpy as np
import pickle
from pandas.core.algorithms import isin

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

### Zadanie 1. Znajdź listę wszystkich filmów o tej samej długości.

In [2]:
length = 84

zad1 = f'''SELECT
                f.title,
                f.length
            FROM
                film f 
            WHERE
                f.length = {length}'''

pd.read_sql(zad1, con=connection)

,title,length
0,Daisy Menagerie,84
1,Gone Trouble,84
2,Guys Falcon,84
3,High Encino,84
4,Honey Ties,84
5,Idols Snatchers,84
6,Island Exorcist,84
7,Jeepers Wedding,84
8,Packer Madigan,84
9,Pluto Oleander,84


### Zadanie 2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
city = 'Abu Dhabi'
zad2 = f'''SELECT
                c.city,
                cst.first_name,
                cst.last_name
            FROM
                customer cst
                    INNER JOIN address a ON cst.address_id = a.address_id
                    INNER JOIN city c ON a.city_id = c.city_id
            WHERE
                c.city = '{city}' '''

pd.read_sql(zad2, con=connection)

,city,first_name,last_name
0,Abu Dhabi,Tom,Milner


### Zadanie 3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
zad3 = f'''SELECT
                AVG(amount)
            FROM
                payment'''

pd.read_sql(zad3, con=connection)

,avg
0,4.200606


### Zadanie 4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
zad4 = f'''SELECT
                c.name as category,
                COUNT(f.*)
            FROM film f
                INNER JOIN film_category fc ON f.film_id = fc.film_id
                INNER JOIN category c ON c.category_id = fc.category_id
            GROUP BY
                c.name'''

pd.read_sql(zad4, con=connection)            

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


### Zadanie 5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
zad5 = f'''SELECT
                c.country,
                COUNT(cst.*)
            FROM customer cst
                INNER JOIN address a ON cst.address_id = a.address_id
                INNER JOIN city ON city.city_id = a.city_id
                INNER JOIN country c ON c.country_id = city.country_id
            GROUP BY
                c.country_id'''

pd.read_sql(zad5, con=connection)   

,country,count
0,"Virgin Islands, U.S.",1
1,Hungary,1
2,Bahrain,1
3,Russian Federation,28
4,Greece,2
...,...,...
103,Pakistan,5
104,Holy See (Vatican City State),1
105,Philippines,20
106,Anguilla,1


### Zadanie 6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
zad6 = f'''SELECT 
                store.store_id,  
                a.address,
                staff.first_name as menager_first_name,
                staff.last_name as menager_last_name,
                COUNT(c.*)
            FROM 
                store
                    INNER JOIN address a ON store.address_id = a.address_id
                    INNER JOIN staff ON store.manager_staff_id = staff.staff_id
                    INNER JOIN customer c ON store.store_id = c.store_id
                    
            GROUP BY 
                store.store_id,  
                a.address,
                staff.first_name,
                staff.last_name 
            HAVING 
                COUNT(c.*) BETWEEN 100 AND 300'''
 
pd.read_sql(zad6, con=connection)   

,store_id,address,menager_first_name,menager_last_name,count
0,2,28 MySQL Boulevard,Jon,Stephens,273


### Zadanie 7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
zad7 = f'''SELECT 
                cst.first_name,
                cst.last_name,
                SUM(f.length)
            FROM
                customer cst
                    INNER JOIN rental r ON cst.customer_id = r.customer_id 
                    INNER JOIN inventory i ON i.inventory_id = r.inventory_id
                    INNER JOIN film f ON f.film_id = i.film_id
                    
            GROUP BY
                cst.first_name,
                cst.last_name
            HAVING
                SUM(f.length) > 200'''

pd.read_sql(zad7, con=connection)

,first_name,last_name,sum
0,Scott,Shelley,2916
1,Tim,Cary,4476
2,Samuel,Marlow,2291
3,Vera,Mccoy,2275
4,Nelson,Christenson,2119
...,...,...,...
594,Erica,Matthews,2727
595,Stanley,Scroggins,3694
596,Ellen,Simpson,2977
597,Patsy,Davidson,3513


### Zadanie 8. Oblicz średnią wartość wypożyczenia filmu.


In [9]:
zad8 = f'''SELECT
                f.title,
                AVG(p.amount)
            FROM
                film f
                    INNER JOIN inventory i ON i.film_id = f.film_id
                    INNER JOIN rental r ON r.inventory_id = i.inventory_id
                    INNER JOIN payment p ON r.rental_id = p.rental_id
                    
            GROUP BY
                f.title'''

pd.read_sql(zad8, con=connection)

,title,avg
0,Frontier Cabin,5.990000
1,Arachnophobia Rollercoaster,4.772609
2,Cruelty Unforgiven,1.156667
3,Intolerable Intentions,5.704286
4,Monsoon Cause,6.101111
...,...,...
953,Reef Salute,2.171818
954,Bugsy Song,5.101111
955,Orient Closer,4.754706
956,Brotherhood Blanket,4.294348


### Zadanie 9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
zad9 = f'''SELECT
                c.name as category,
                AVG(f.length)
            FROM film f
                INNER JOIN film_category fc ON f.film_id = fc.film_id
                INNER JOIN category c ON c.category_id = fc.category_id
            GROUP BY
                c.name'''
pd.read_sql(zad9, con=connection)

,category,avg
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


### Zadanie 10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
zad10 = f'''SELECT
                *
            FROM    
                (SELECT
                    cat.name,
                    MAX(f.length) as max_length
                FROM
                    film f
                        INNER JOIN film_category fc ON f.film_id = fc.film_id
                        INNER JOIN category cat ON fc.category_id = cat.category_id
                GROUP BY
                    cat.name) temp_1
            JOIN
                (SELECT
                    cat.name,
                    f.length as max_length,
                    f.title
                FROM film f
                        INNER JOIN film_category fc ON f.film_id = fc.film_id
                        INNER JOIN category cat ON fc.category_id = cat.category_id) temp2
                        
            USING(name, max_length)'''

pd.read_sql(zad10, con=connection)

,name,max_length,title
0,Horror,181,Analyze Hoosiers
1,Games,185,Chicago North
2,Classics,184,Conspiracy Spirit
3,Comedy,185,Control Anthem
4,Foreign,184,Crystal Breaking
5,Action,185,Darn Forrester
6,New,183,Frontier Cabin
7,Children,178,Fury Murder
8,Animation,185,Gangs Pride
9,Music,185,Home Pity


### Zadanie 11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

# Test zadań implementacyjnych

In [12]:
m.film_in_category(4)

,title,languge,category
0,Alice Fantasia,English,Classics
1,Arizona Bang,English,Classics
2,Beast Hunchback,English,Classics
3,Bound Cheaper,English,Classics
4,Candidate Perdition,English,Classics
5,Center Dinosaur,English,Classics
6,Color Philadelphia,English,Classics
7,Conspiracy Spirit,English,Classics
8,Core Suit,English,Classics
9,Creepers Kane,English,Classics


In [13]:
m.number_films_in_category(4)

,category,count
0,Classics,57


In [14]:
m.number_film_by_length(120,130)

,length,count
0,120,9
1,121,8
2,122,11
3,123,7
4,124,4
5,125,7
6,126,9
7,127,5
8,128,7
9,129,7


In [15]:
m.client_from_city('Athenai')

,city,first_name,last_name
0,Athenai,Linda,Williams


In [16]:
m.avg_amount_by_length(48)

,length,avg
0,48,4.295389


In [17]:
m.client_by_sum_length(5000)

,first_name,last_name,sum
0,Tammy,Sanders,5065
1,Eleanor,Hunt,5360
2,Karl,Seal,5388


In [18]:
m.category_statistic_length('Action')

,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
